In [ ]:
# IMPORTS
import pandas as pd
import sqlite3
import re
import math

In [ ]:
UC_DATASET_DIR             = f"./assets"
PIB_DATASET_DIR            = f"{ASSETS_DIR}/PIB_por_ano.csv"
POPULACAO_DATASET_DIR      = f"{ASSETS_DIR}/estimativa_populacao.csv"
SALARIOS_DATASET_DIR       = f"{ASSETS_DIR}/massa_salarial_por_ano.csv"
# EMPREGOS_DATASET_DIR       = f"{ASSETS_DIR}"
# TOTAL_EMPRESAS_DATASET_DIR = f"{ASSETS_DIR}"